In [1]:
%load_ext autoreload
%autoreload 2
%cd /mnt/c/Users/resha/Documents/Github/balancing_framework/

import os
import pickle
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd

import time
import optuna
import argparse


from gluonts.evaluation import make_evaluation_predictions
from gluonts.mx.trainer import Trainer
from gluonts.mx.trainer.callback import TrainingHistory
from gluonts.mx.distribution import StudentTOutput, MultivariateGaussianOutput
from sklearn.metrics import mean_absolute_error
from gluonts.dataset.multivariate_grouper import MultivariateGrouper

from gluonts.mx.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.mx.model.transformer import TransformerEstimator
from gluonts.mx.model.deepar import DeepAREstimator 
from gluonts.mx.model.wavenet import WaveNetEstimator
from gluonts.mx.model.seq2seq import MQCNNEstimator

import matplotlib.pyplot as plt
import json
from pathlib import Path
from gluonts.dataset.split import split
from gluonts.dataset.common import ListDataset
import copy

from gluonts.evaluation import Evaluator
from gluonts.evaluation import make_evaluation_predictions

from gluonts_utils import series_to_gluonts_dataset, load_params

class Objective:
    def __init__( self, model, dataset_name, X):
        self.model = model
        self.dataset_name = dataset_name
        self.data_params = load_params('gluonts_params.txt', dataset_name)
        print(self.data_params)
        self.ctx = 'gpu(0)'

        self.prediction_length = self.data_params['prediction_length']
        self.context_length = self.data_params['context_length']
        self.freq = self.data_params['freq']
        
        X_train_val, X_test = X[:-self.prediction_length], X[-self.prediction_length:]
        X_train, X_val = X_train_val[:-self.prediction_length], X_train_val[-self.prediction_length:]
        
        self.tuning_dataset = series_to_gluonts_dataset(X_train, X_val,  self.data_params)
        self.eval_dataset =  series_to_gluonts_dataset(X_train_val, X_test,  self.data_params)
        

    def get_params(self, trial) -> dict:
        if self.model == 'feedforward':
            return {
              "num_hidden_dimensions": [trial.suggest_int("hidden_dim_{}".format(i), 10, 100) for i in range(trial.suggest_int("num_layers", 1, 5))],
              "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
              "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
            }
        elif self.model == 'wavenet':
            return {
                "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
                "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
            }
        elif self.model == 'mqcnn':
            return {
                "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
                "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
            }
        elif self.model == 'deepar':
            return {
                "num_cells": trial.suggest_int("num_cells", 10, 100),
                "num_layers": trial.suggest_int("num_layers", 1, 10),
                "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
                "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100)
            }
        elif self.model == 'transformer':
            # num_heads must divide model_dim
            valid_pairs = [ (i,d) for i in range(10,101) for d in range(1,11) if i%d == 0  ]
            model_dim_num_heads_pair = trial.suggest_categorical("model_dim_num_heads_pair", valid_pairs)

            return {
                "inner_ff_dim_scale": trial.suggest_int("inner_ff_dim_scale", 1, 5),
                "model_dim": model_dim_num_heads_pair[0],
                "embedding_dimension": trial.suggest_int("embedding_dimension", 1, 10),
                "num_heads": model_dim_num_heads_pair[1],
                "dropout_rate": trial.suggest_uniform("dropout_rate", 0.0, 0.5),
                "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
                "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
            }
        
    def load_model(self, params):
        history = TrainingHistory()
        if self.model == 'feedforward':
            estimator = SimpleFeedForwardEstimator(
                num_hidden_dimensions= params['num_hidden_dimensions'], #num_hidden_dimensions,
                prediction_length=self.prediction_length,
                context_length=self.context_length,
                batch_normalization=False,
                mean_scaling=False,
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                num_batches_per_epoch=100, callbacks=[history]),
            )
        elif self.model == 'wavenet':
            estimator = WaveNetEstimator(
                freq=self.freq,
                prediction_length=self.prediction_length,
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                    num_batches_per_epoch=100, callbacks=[history], add_default_callbacks=False),
            )
        elif self.model == 'mqcnn':
            estimator = MQCNNEstimator(
                freq=self.freq,
                prediction_length=self.prediction_length,
                context_length=self.context_length,
                distr_output=StudentTOutput(),
                quantiles=None,
                scaling=False, 
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                num_batches_per_epoch=100, callbacks=[history], hybridize=False),
            )
        elif self.model == 'deepar':
            estimator = DeepAREstimator(
                freq=self.freq,
                context_length=self.context_length,
                distr_output=StudentTOutput(),
                prediction_length=self.prediction_length,
                # num_cells= params['num_cells'],
                # num_layers= params['num_layers'],
                scaling=False, # True by default
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                num_batches_per_epoch=100, callbacks=[history]),
            )
        elif self.model == 'transformer':
            estimator = TransformerEstimator(
                freq=self.freq,
                context_length=self.context_length,
                prediction_length=self.prediction_length,
                distr_output=StudentTOutput(),
                inner_ff_dim_scale= params['inner_ff_dim_scale'],
                model_dim= params['model_dim'],
                embedding_dimension= params['embedding_dimension'],
                num_heads= params['num_heads'],
                dropout_rate= params['dropout_rate'],
                # scaling=False, # True by default False
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                num_batches_per_epoch=100, callbacks=[history]),
            )

        return estimator, history

    def train_test(self, params, tuning=True):
        model, history = self.load_model(params)

        if tuning:
            predictor = model.train(self.tuning_dataset.train, self.tuning_dataset.test)
            forecast_it, ts_it = make_evaluation_predictions(
                dataset=self.tuning_dataset.test,
                predictor=predictor,
            )
        else:
            predictor = model.train(self.eval_dataset.train)
            forecast_it, ts_it = make_evaluation_predictions(
                dataset=self.eval_dataset.test,
                predictor=predictor,
            )

        forecasts = list(forecast_it)
        tss = list(ts_it)
        evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
        agg_metrics, item_metrics = evaluator(tss, forecasts)
        
        print(f'#####__tuning mase = {agg_metrics["MASE"]}__###### ')
        return agg_metrics, predictor, history
        

    def __call__(self, trial):

        params = self.get_params(trial)

        agg_metrics, _, _ = self.train_test(params, tuning=True)

        return agg_metrics['MASE']

/mnt/c/Users/resha/Documents/Github/balancing_framework


In [2]:
X = pd.read_csv('m4_1165_fd.csv') # original fd, default thresh, no skips
X_withoutfd = X.drop(columns=['values_fd'])
X_fdtarget = X.drop(columns=['values_o']).rename(columns={"values_fd": "values_o"})

In [8]:

def run(X, model, dataset_name, save_label, n_trials, n_repeats):
    start_time = time.perf_counter()

    # run tuning
    study = optuna.create_study(direction="minimize")
    obj = Objective(
        model=model,
        dataset_name=dataset_name,
        X=X
        )
    study.optimize(
        obj,
        n_trials=n_trials,
    )
    trial = study.best_trial

    # print results
    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    print(f'Runtime: {time.perf_counter() - start_time}')

    # unpack params for next runs
    if model == 'feedforward':
        trial.params["num_hidden_dimensions"] = [ trial.params[f"hidden_dim_{i}"] for i in range(trial.params["num_layers"]) ]
    elif model == 'transformer':
        trial.params["model_dim"] = trial.params["model_dim_num_heads_pair"][0]
        trial.params["num_heads"] = trial.params["model_dim_num_heads_pair"][1]

    # repeat best run 5 times
    mases = []
    smapes = []
    rmses = []
    params_sets = []
    save_dir = f'results/monash_gluonts_single_tuned_runs/{model}_{dataset_name}_{save_label}_{n_trials}_trials'
    os.makedirs(save_dir, exist_ok=True)
    for i in range(n_repeats):
        res, predictor, history = obj.train_test(trial.params, tuning=False)

        # plot and save training history
        plt.plot(history.loss_history, label='Training Loss')
        plt.plot(history.validation_loss_history, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Learning Curve')
        plt.legend()
        plt.grid(True)

        # Save the figure
        plt.savefig(f'{save_dir}/learning_curve_{i}.png')
        # save the history values
        with open(f'{save_dir}/loss_history_{i}.json', "w") as f:
            json.dump(history.loss_history, f)
        # Clear the current figure
        plt.clf()
        mases.append(res['MASE'])
        smapes.append(res['sMAPE'])
        rmses.append(res['RMSE'])
        params_sets.append(trial.params)


    mase_mean = np.array(mases).mean()
    mase_std = np.std(np.array(mases))
    rmses_mean = np.array(rmses).mean()
    rmses_std = np.std(np.array(rmses))
    smape_mean = np.array(smapes).mean()
    smape_std = np.std(np.array(smapes))

    print(f'##### MASE MEAN: {mase_mean} MASE STD: {mase_std}')
    print(f'##### sMAPE MEAN: {smape_mean} sMAPE STD: {smape_std}')
    print(f'##### RMSE MEAN: {rmses_mean} RMSE STD: {rmses_std}')


    # save best params to json
    with open(f'{save_dir}/params.json', "w") as f:
        json.dump(trial.params, f)

    # save the last predictor
    os.makedirs(f'{save_dir}/predictor', exist_ok=True)
    predictor.serialize(Path(f"{save_dir}/predictor"))

    end_time = time.perf_counter()
    runtime = (end_time - start_time) / 60

    file_path = "gluonts_results.txt"
    with open(file_path, "a") as file:
        file.write(
            f'''\n\n\n\n {model} {n_trials} tuning trials {n_repeats} repeat test evals on {dataset_name} {save_label} data form, Runtime: {runtime} minutes 
            \n Params: {trial.params}
            \n MASE MEAN: {mase_mean} MASE STD: {mase_std}
            \n sMAPE MEAN: {smape_mean} sMAPE STD: {smape_std}
            \n RMSE MEAN: {rmses_mean} RMSE STD: {rmses_std}
            '''
        )

[I 2025-09-17 10:11:00,514] A new study created in memory with name: no-name-8dd33731-739f-415e-8d33-ed7562090f6f


{'freq': 'D', 'prediction_length': 140, 'context_length': 90}


100%|██████████| 100/100 [00:26<00:00,  3.78it/s, epoch=1/69, avg_epoch_loss=13.5]
1it [00:00, 22.16it/s, epoch=1/69, validation_avg_epoch_loss=14.6]
100%|██████████| 100/100 [00:13<00:00,  7.28it/s, epoch=2/69, avg_epoch_loss=13.4]
1it [00:00, 25.12it/s, epoch=2/69, validation_avg_epoch_loss=14.4]
100%|██████████| 100/100 [00:13<00:00,  7.46it/s, epoch=3/69, avg_epoch_loss=13.2]
1it [00:00, 24.11it/s, epoch=3/69, validation_avg_epoch_loss=14.2]
100%|██████████| 100/100 [00:13<00:00,  7.35it/s, epoch=4/69, avg_epoch_loss=13.1]
1it [00:00, 24.41it/s, epoch=4/69, validation_avg_epoch_loss=14]
100%|██████████| 100/100 [00:13<00:00,  7.49it/s, epoch=5/69, avg_epoch_loss=13]
1it [00:00, 26.83it/s, epoch=5/69, validation_avg_epoch_loss=13.8]
100%|██████████| 100/100 [00:13<00:00,  7.52it/s, epoch=6/69, avg_epoch_loss=12.9]
1it [00:00, 27.15it/s, epoch=6/69, validation_avg_epoch_loss=13.6]
100%|██████████| 100/100 [00:13<00:00,  7.33it/s, epoch=7/69, avg_epoch_loss=12.7]
1it [00:00, 21.47it/s

#####__tuning mase = 111.25625362426327__###### 


100%|██████████| 100/100 [00:14<00:00,  6.75it/s, epoch=1/94, avg_epoch_loss=10.6]
1it [00:00, 28.81it/s, epoch=1/94, validation_avg_epoch_loss=10.3]
100%|██████████| 100/100 [00:13<00:00,  7.42it/s, epoch=2/94, avg_epoch_loss=10.3]
1it [00:00, 23.17it/s, epoch=2/94, validation_avg_epoch_loss=9.9]
100%|██████████| 100/100 [00:13<00:00,  7.31it/s, epoch=3/94, avg_epoch_loss=10.1]
1it [00:00, 27.84it/s, epoch=3/94, validation_avg_epoch_loss=9.62]
100%|██████████| 100/100 [00:14<00:00,  6.94it/s, epoch=4/94, avg_epoch_loss=9.9]
1it [00:00, 27.84it/s, epoch=4/94, validation_avg_epoch_loss=9.46]
100%|██████████| 100/100 [00:14<00:00,  6.93it/s, epoch=5/94, avg_epoch_loss=9.7]
1it [00:00, 27.05it/s, epoch=5/94, validation_avg_epoch_loss=9.27]
100%|██████████| 100/100 [00:14<00:00,  7.08it/s, epoch=6/94, avg_epoch_loss=9.53]
1it [00:00, 28.47it/s, epoch=6/94, validation_avg_epoch_loss=9.26]
100%|██████████| 100/100 [00:14<00:00,  7.05it/s, epoch=7/94, avg_epoch_loss=9.35]
1it [00:00, 28.89it/

...

#####__tuning mase = 28.2304914521156__###### 


100%|██████████| 100/100 [00:15<00:00,  6.29it/s, epoch=1/90, avg_epoch_loss=9.99]
1it [00:00, 24.23it/s, epoch=1/90, validation_avg_epoch_loss=8.71]
100%|██████████| 100/100 [00:14<00:00,  6.73it/s, epoch=2/90, avg_epoch_loss=8.57]
1it [00:00, 23.52it/s, epoch=2/90, validation_avg_epoch_loss=7.9]
100%|██████████| 100/100 [00:15<00:00,  6.45it/s, epoch=3/90, avg_epoch_loss=7.89]
1it [00:00, 22.06it/s, epoch=3/90, validation_avg_epoch_loss=7.46]
100%|██████████| 100/100 [00:15<00:00,  6.46it/s, epoch=4/90, avg_epoch_loss=7.68]
1it [00:00, 27.22it/s, epoch=4/90, validation_avg_epoch_loss=7.22]
100%|██████████| 100/100 [00:14<00:00,  6.86it/s, epoch=5/90, avg_epoch_loss=7.48]
1it [00:00, 25.94it/s, epoch=5/90, validation_avg_epoch_loss=7.37]
100%|██████████| 100/100 [00:14<00:00,  6.77it/s, epoch=6/90, avg_epoch_loss=7.36]
1it [00:00, 23.57it/s, epoch=6/90, validation_avg_epoch_loss=7.28]
100%|██████████| 100/100 [00:14<00:00,  6.97it/s, epoch=7/90, avg_epoch_loss=7.22]
1it [00:00, 28.86i

#####__tuning mase = 2.039018232606041__###### 


100%|██████████| 100/100 [00:16<00:00,  6.20it/s, epoch=1/94, avg_epoch_loss=9.82]
1it [00:00, 25.57it/s, epoch=1/94, validation_avg_epoch_loss=8.64]
100%|██████████| 100/100 [00:13<00:00,  7.24it/s, epoch=2/94, avg_epoch_loss=8.36]
1it [00:00, 28.60it/s, epoch=2/94, validation_avg_epoch_loss=7.8]
100%|██████████| 100/100 [00:13<00:00,  7.39it/s, epoch=3/94, avg_epoch_loss=7.79]
1it [00:00, 31.15it/s, epoch=3/94, validation_avg_epoch_loss=7.13]
100%|██████████| 100/100 [00:13<00:00,  7.44it/s, epoch=4/94, avg_epoch_loss=7.58]
1it [00:00, 29.42it/s, epoch=4/94, validation_avg_epoch_loss=7.26]
100%|██████████| 100/100 [00:13<00:00,  7.21it/s, epoch=5/94, avg_epoch_loss=7.42]
1it [00:00, 25.35it/s, epoch=5/94, validation_avg_epoch_loss=8.58]
100%|██████████| 100/100 [00:13<00:00,  7.27it/s, epoch=6/94, avg_epoch_loss=7.28]
1it [00:00, 28.13it/s, epoch=6/94, validation_avg_epoch_loss=6.92]
100%|██████████| 100/100 [00:13<00:00,  7.22it/s, epoch=7/94, avg_epoch_loss=7.22]
1it [00:00, 28.79i

: 

pred 14

##### MASE MEAN: 3.6232407327089247 MASE STD: 0.19377496086625648
##### sMAPE MEAN: 0.016397337828363688 sMAPE STD: 0.0008688977288584552

pred 28?

##### MASE MEAN: 1.7459754530010099 MASE STD: 0.07188842725219177
##### sMAPE MEAN: 0.007905414061886926 sMAPE STD: 0.00032358597034601273

pred 28 fd original no skip

##### MASE MEAN: 2.4670366780837263 MASE STD: 0.9511033219491887 # ignore without inversion
##### sMAPE MEAN: 0.029764325916767122 sMAPE STD: 0.01188769563227722